# About: VCノードの起動

---

VCノードを起動します。

## グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

> グループ名を指定することにより「01_パラメータ設定.ipynb」で設定した値を引き継ぐことが出来ます。

In [1]:
target_group = 'Moodle'

# 前提条件の確認

事前にNotebook「[01_パラメータ設定](01_パラメータ設定.ipynb)」が実行されていて、パラメータを記録したファイルが存在することを確認します。

In [2]:
import os
if not os.path.exists(os.path.join('group_vars', target_group + '.yml')):
    raise RuntimeError("ERROR: not exists {}".format(target_group + '.yml'))

# VCノードの起動

VCP SDK を利用してVCノードを起動します。

![VCノードの起動](image/moodle-2-03.png)

## VCP SDKの初期化とパラメータの指定

VCP SDKを初期化した後、VCノードのパラメータ設定を行いVCノードを起動します。設定する主なパラメータを以下に示します。

* ユーザ名(必須項目):
    - spec.params_e[ANSIBLE_USER] 
* SSH公開鍵(必須項目):
    - spec.params_e[SSH_PUBLIC_KEY] 
* VCノードにアタッチするEBSの情報(オプション):
    - spec.volume_id
* VCノードのIPアドレス(オプション):
    - spec.ip_address_list 
* VCノードのインスタンスタイプ(オプション):
    - spec.instance_type
* VCノードのルートボリュームサイズ(オプション):
    - spec.volume_size

以下のセルで実際の操作を行っていきます。

まずVCP SDK の初期化を行います。

In [3]:
from common import logsetting
from vcpsdk import vcpsdk
%run scripts/group.py

gvars = load_group_vars(target_group)
sdk = vcpsdk.VcpSDK(
    gvars['vcc_access_token'],
    target_group)

VCノードにSSHでログインするためのユーザ名と公開鍵の指定を行います。

In [4]:
import base64

spec = sdk.spec.find_spec(
    gvars['vcp_provider'], gvars['vcp_flavor'])

with open(gvars['ansible_ssh_public_key_file'], 'rb') as f:
    b64_ssh_publickey = base64.b64encode(f.read()).decode('UTF-8')

spec.params_e = [
    "ANSIBLE_USER={}".format(gvars['ansible_user']),  # ユーザ名の指定
    "SSH_PUBLIC_KEY={}".format(b64_ssh_publickey),    # 公開鍵の指定
]

VCノードにアタッチするMySQL, Moodle用のボリューム(EBS)に関する指定を行います。

In [5]:
# ボリューム(MySQL, Moodle)をVCノードにアタッチ
vcp_provider = gvars['vcp_provider']
if  vcp_provider == 'aws':
    spec.volume_id = [
        gvars['volume_database_id'],
        gvars['volume_moodle_id'],
    ]
elif vcp_provider == 'azure':
    spec.volume_id = [{
        'id': gvars['volume_database_id'],
        'size_gb': int(gvars['volume_database_size']),
    },{
        'id': gvars['volume_moodle_id'],
        'size_gb': int(gvars['volume_moodle_size']),
    }]

# ボリューム暗号化の有効、無効の指定
if gvars['volume_encrypt']:       # 暗号化する場合
    spec.image = 'vcp-moodle:luks'
    spec.params_e.extend([
        # Vaultサーバのアドレス
        "VAULT_ADDR={}".format(gvars['vault_address']),
        # Vaultサーバのアクセストークン
        "VAULT_TOKEN={}".format(gvars['vcc_access_token']),
        # 暗号化の鍵ファイルの格納場所
        "LUKS_KEYFILE_PATH={}".format(gvars['vault_luks_key_path']),
        # 暗号化対象となるボリュームのデバイス名のマッピング情報
        "LUKS_ENTRIES={}={}:{}={}".format(
            'mysql', gvars['volume_database_device0'],
            'moodle', gvars['volume_moodle_device0'],
        ),
        # ハンズオンではEBSを使いまわしているので強制的にフォーマットする
        "LUKS_FORCE_FORMAT=YES",
    ])
else:                              # 暗号化しない場合
    spec.image = 'vcp-moodle:base'

VCノードに割り当てるIPアドレスを指定します。

In [6]:
if 'vc_node_address' in gvars:
    spec.ip_address_list = [gvars['vc_node_address']]

その他のパラメータを指定します。

In [7]:
if 'vcp_instance_type' in gvars:
    # インスタンスタイプの指定
    spec.instance_type = gvars['vcp_instance_type']
    
if 'root_volume_size' in gvars:
    # ルートボリュームサイズの指定
    spec.volume_size = gvars['root_volume_size']
    
# Docker volume の指定
spec.params_v = ['{0}:{0}'.format(gvars['project_dir'])]

## VCノードの起動

VCノードを起動します。

> 起動には1分半程度の時間がかかります。

In [8]:
nodes = sdk.unit.create(
    '{}-server'.format(target_group), spec, wait_for=True, verbose=0)

2018-06-21 16:05:15,141 - INFO - BOOTING ... 0 sec
2018-06-21 16:05:20,418 - INFO - BOOTING ... 5 sec
2018-06-21 16:05:25,730 - INFO - BOOTING ... 10 sec
2018-06-21 16:05:31,007 - INFO - BOOTING ... 15 sec
2018-06-21 16:05:36,303 - INFO - BOOTING ... 20 sec
2018-06-21 16:05:41,587 - INFO - BOOTING ... 25 sec
2018-06-21 16:05:46,878 - INFO - BOOTING ... 30 sec
2018-06-21 16:05:52,168 - INFO - BOOTING ... 35 sec
2018-06-21 16:05:57,475 - INFO - BOOTING ... 40 sec
2018-06-21 16:06:02,778 - INFO - BOOTING ... 45 sec
2018-06-21 16:06:08,149 - INFO - BOOTING ... 50 sec
2018-06-21 16:06:13,497 - INFO - BOOTING ... 55 sec
2018-06-21 16:06:18,829 - INFO - BOOTING ... 60 sec
2018-06-21 16:06:24,143 - INFO - BOOTING ... 65 sec
2018-06-21 16:06:29,465 - INFO - BOOTING ... 70 sec
2018-06-21 16:06:34,787 - INFO - BOOTING ... 75 sec
2018-06-21 16:06:40,137 - INFO - BOOTING ... 80 sec
2018-06-21 16:06:45,430 - INFO - unit Moodle-server is RUNNING


作成されたVCの状態を確認します。

In [9]:
sdk.unit.df_node()

,vcno,vc_name,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,23,Moodle,Moodle-server,RUNNING,1,68f4d05a...,RUNNING,172.30.2.100,i-0e1136c5bfb414df8,VCP-2045b119-09bf021c,exists


VCノードの状態が `RUNNING` になっていることを確認します。

> VCノードの起動に失敗して`RUNNING`以外の状態になっている場合は次のセルを実行するとエラーになります。

In [10]:
if nodes[0].unit.nodes[0].state != 'RUNNING':
    raise RuntimeError('ERROR: not running')

VCノードのIPアドレスを確認します。

In [11]:
bc_ip_address = [
    n.cloud_instance_address
    if isinstance(n.cloud_instance_address, str)
    else n.cloud_instance_address.encode('utf-8')
    for n in nodes][0]

if 'vc_node_address' in gvars:
    # 指定したIPアドレスと一致していることを確認する
    if bc_ip_address != gvars['vc_node_address']:
        raise RuntimeError(
            "ERROR: {} != {}".format(bc_ip_address, gvars['vc_node_address']))

print(bc_ip_address)

172.30.2.100


# Ansibleの設定

起動したVCノードをAnsibleのインベントリに登録します。またAnsibleでVCノードに接続できることを確認します。

![Ansibleの設定](image/moodle-2-04.png)

## インベントリの更新

インベントリを更新するための作業ディレクトリを作成します。

In [12]:
import tempfile
work_dir = tempfile.mkdtemp()

編集前のインベントリを作業ディレクトリにコピーします。

In [13]:
import os
!touch hosts
!cp hosts {os.path.join(work_dir, 'hosts.orig')}

VCノードのIPアドレスをインベントリに追加します。

In [14]:
arg = '''
path=hosts marker="# {mark} ANSIBLE MANAGED BLOCK {{target_group}}"
block="[{{target_group}}]
{{vc_node_address}}
"
'''

!ansible --extra-vars="@group_vars/{target_group}.yml" \
    -c local -m blockinfile -a '{arg}' localhost

# 更新後のインベントリを表示する 
!cat hosts

 [WARNING]: Could not match supplied host pattern, ignoring: all

 [WARNING]: provided hosts list is empty, only localhost is available

localhost | SUCCESS => {
    "changed": true, 
    "failed": false, 
    "msg": "Block inserted"
}
# BEGIN ANSIBLE MANAGED BLOCK Moodle
[Moodle]
172.30.2.100
# END ANSIBLE MANAGED BLOCK Moodle

インベントリを更に編集したい場合は、次のセルを実行して表示されるリンクから編集画面を開いて編集してください。

In [15]:
%run scripts/nbutils.py
nb_html_link(os.path.join(os.getcwd(), 'hosts'), 'Inventory')

更新前のインベントリとの差分を確認します。

> 更新前のファイルとの差分がない場合はVCノードが登録されていないとみなして、次のセルがエラーになります。

In [16]:
try:
    !diff -u {os.path.join(work_dir, 'hosts.orig')} hosts
    raise Exception("WARNING: no change!")
except RuntimeError:
    pass

--- /tmp/tmp215pvcei/hosts.orig	2018-06-21 16:07:05.213230023 +0900
+++ hosts	2018-06-21 16:07:06.569232000 +0900
@@ -0,0 +1,4 @@
+# BEGIN ANSIBLE MANAGED BLOCK Moodle
+[Moodle]
+172.30.2.100
+# END ANSIBLE MANAGED BLOCK Moodle
\ No newline at end of file


Ansibleの対象グループに登録されているホストの一覧を表示して、追加したVCノードのIPアドレスが表示されることを確認します。

> ハンズオン環境では `172.30.2.100` が表示されることを確認してください。

In [17]:
!ansible --list-hosts {target_group}

  hosts (1):
    172.30.2.100


## VCノードとの接続確認

VCノードに対して Ansible で接続できることを確認します。

In [18]:
def reachable_check():
    !ansible -m ping {target_group}


%run scripts/utils.py
retry_exec(reachable_check)

172.30.2.100 | SUCCESS => {
    "changed": false, 
    "failed": false, 
    "ping": "pong"
}


正常に接続できると以下のように表示されます。

```
XXX.XXX.XXX.XXX | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
```

タイミングによってはまだVCノードのsshdサービスが開始されておらず `UNREACHABLE` となってしまう場合もあります。そのため何度かリトライして確認しています。

SSHの設定によっては以下のように表示されることがあります。
```
The authenticity of host '[XXX.XXX.XXX.XXX]:xxxxx' can't be established.
RSA key fingerprint is xx:xx:xx:xx:xx:xx:xx:xx:xx:xx:xx:xx:xx:xx:xx:xx.
Are you sure you want to continue connecting (yes/no)? 
```
この場合はNotebookのメニューから[Kernel]-[Interrupt]を選択してセルの実行を一旦中断させてください。sshがユーザの入力を要求しているので Terminal で接続確認を行ってください。

VCノードに対して設定ファイルの変更やパッケージの追加を行う場合にVCノードの管理者権限が必要になる場合があります。Ansibleで管理者権限によるコマンド実行が可能かどうかを確認します。

In [19]:
# 管理者権限(-b)でのコマンド実行
!ansible -b -a 'whoami' {target_group}

172.30.2.100 | SUCCESS | rc=0 >>
root



インベントリの更新作業が完了したので作業用ディレクトリを削除します。

In [20]:
!rm -fr {work_dir}

# データ格納領域の準備

![データ格納領域の準備](image/moodle-2-05.png)

Moodle用, MySQL用のボリュームをデータを格納する領域として利用できるように準備します。



主な操作を以下に記します。

* ファイルシステムの作成：
```
# mkfs.btfs -L {label名} /dev/{デバイス名} 
```
* ファイルシステムのマウント：
```
# （ansibleのmountコマンドを利用）
```
* ディレクトリ（サブボリューム）の作成：
```
# btrfs subvolume create {対象ディレクトリ}
```

## デバイスにアクセスできることの確認

Moodle用, MySQL用のデバイスにアクセスできるようになるまで待ち合わせを行います。

> ボリュームの暗号化を有効にしている場合は、VCノードを起動してから暗号化の初期処理が完了するまで数分程度を要します。

In [21]:
%run scripts/utils.py

def check_device_access():
    !ansible -a 'ls {{{{volume_moodle_device}}}}' {target_group}
    !ansible -a 'ls {{{{volume_database_device}}}}' {target_group}
    
retry_exec(check_device_access)

172.30.2.100 | SUCCESS | rc=0 >>
/dev/mapper/moodle

172.30.2.100 | SUCCESS | rc=0 >>
/dev/mapper/mysql



VCノードのブロックデバイスを確認します。ボリュームの暗号化を有効にしている場合は TYPE が `crypt` となっているエントリが表示されます。

In [22]:
!ansible -a 'lsblk' {target_group}

172.30.2.100 | SUCCESS | rc=0 >>
NAME     MAJ:MIN RM SIZE RO TYPE  MOUNTPOINT
xvda     202:0    0  20G  0 disk  
└─xvda1  202:1    0  20G  0 part  /var/lib/docker/aufs
xvdf     202:80   0   8G  0 disk  
└─mysql  252:1    0   8G  0 crypt 
xvdg     202:96   0   8G  0 disk  
└─moodle 252:0    0   8G  0 crypt 



## ファイルシステムの作成

### Moodle用ボリュームのファイルシステム作成

mkfs コマンドを実行してファイルシステムを作成します。

> オプションを指定する場合は次のセルの `mkfs_opts` に値を設定してください。

In [23]:
# mkfs のオプションを指定する
mkfs_opts = '-f'   # force オプションの指定

!ansible -b -a 'mkfs.{{{{volume_moodle_fs}}}} \
    -L {{{{volume_moodle_fs_label}}}} {mkfs_opts} \
    {{{{volume_moodle_device}}}}' {target_group}

172.30.2.100 | SUCCESS | rc=0 >>
btrfs-progs v4.8.5
See http://btrfs.wiki.kernel.org for more information.

Detected a SSD, turning off metadata duplication.  Mkfs with -m dup if you want to force metadata duplication.
Label:              moodle
UUID:               3f387f3e-8559-4224-9418-fde78b8a311b
Node size:          16384
Sector size:        4096
Filesystem size:    8.00GiB
Block group profiles:
  Data:             single            8.00MiB
  Metadata:         single            8.00MiB
  System:           single            4.00MiB
SSD detected:       yes
Incompat features:  extref, skinny-metadata
Number of devices:  1
Devices:
   ID        SIZE  PATH
    1     8.00GiB  /dev/mapper/moodle



作成されたファイルシステムの状態を確認します。

> 今のところ作成するファイルシステムは btrfs のみを前提としているので btrfs コマンドで確認を行います。

In [24]:
if 'btrfs' == load_group_var(target_group, 'volume_moodle_fs'):
    !ansible -b -a 'btrfs filesystem show {{{{volume_moodle_device}}}}' {target_group}

172.30.2.100 | SUCCESS | rc=0 >>
Label: 'moodle'  uuid: 3f387f3e-8559-4224-9418-fde78b8a311b
	Total devices 1 FS bytes used 112.00KiB
	devid    1 size 8.00GiB used 20.00MiB path /dev/mapper/moodle



### MySQL用ボリュームのファイルシステム作成

mkfs コマンドを実行してファイルシステムを作成します。

> オプションを指定する場合は次のセルの `mkfs_opts` に値を設定してください。

In [25]:
# mkfs のオプションを指定する
mkfs_opts = '-f'   # force オプションの指定

!ansible -b -a 'mkfs.{{{{volume_database_fs}}}} \
    -L {{{{volume_database_fs_label}}}} {mkfs_opts} \
    {{{{volume_database_device}}}}' {target_group}

172.30.2.100 | SUCCESS | rc=0 >>
btrfs-progs v4.8.5
See http://btrfs.wiki.kernel.org for more information.

Detected a SSD, turning off metadata duplication.  Mkfs with -m dup if you want to force metadata duplication.
Label:              mysql
UUID:               2db99a78-14a1-4655-92c5-3af138f398c8
Node size:          16384
Sector size:        4096
Filesystem size:    8.00GiB
Block group profiles:
  Data:             single            8.00MiB
  Metadata:         single            8.00MiB
  System:           single            4.00MiB
SSD detected:       yes
Incompat features:  extref, skinny-metadata
Number of devices:  1
Devices:
   ID        SIZE  PATH
    1     8.00GiB  /dev/mapper/mysql



作成されたファイルシステムの状態を確認します。

> 今のところ作成するファイルシステムは btrfs のみを前提としているので btrfs コマンドで確認を行います。

In [26]:
if 'btrfs' == load_group_var(target_group, 'volume_database_fs'):
    !ansible -b -a 'btrfs filesystem show {{{{volume_database_device}}}}' {target_group}

172.30.2.100 | SUCCESS | rc=0 >>
Label: 'mysql'  uuid: 2db99a78-14a1-4655-92c5-3af138f398c8
	Total devices 1 FS bytes used 112.00KiB
	devid    1 size 8.00GiB used 20.00MiB path /dev/mapper/mysql



## ファイルシステムのマウント

作成したファイルシステムをマウントします。同時に /etc/fstab に対応するエントリを追加します。

### Moodle用ボリュームのマウント

Ansible で/etc/fstab とマウント状態の更新を行います。

In [27]:
!ansible -b -m mount -a 'name="{{{{volume_moodle_mount}}}}" \
    src="LABEL={{{{volume_moodle_fs_label}}}}" \
    opts={{{{volume_moodle_mount_opt}}}} \
    fstype={{{{volume_moodle_fs}}}} state=mounted' {target_group}

172.30.2.100 | SUCCESS => {
    "changed": true, 
    "dump": "0", 
    "failed": false, 
    "fstab": "/etc/fstab", 
    "fstype": "btrfs", 
    "name": "/opt/moodle", 
    "opts": "defaults", 
    "passno": "0", 
    "src": "LABEL=moodle"
}


/etc/fstabが更新されたことを確認します。

In [28]:
!ansible -a 'grep LABEL={{{{volume_moodle_fs_label}}}} /etc/fstab' {target_group}

172.30.2.100 | SUCCESS | rc=0 >>
LABEL=moodle /opt/moodle btrfs defaults 0 0



実際にマウントされていることを確認します。

In [29]:
!ansible -m shell -a 'mount | grep -e {{{{volume_moodle_mount}}}}' {target_group}

 [WARNING]: Consider using mount module rather than running mount

172.30.2.100 | SUCCESS | rc=0 >>
/dev/mapper/moodle on /opt/moodle type btrfs (rw,relatime,ssd,space_cache,subvolid=5,subvol=/)



### MySQL用ボリュームのマウント

Ansible で/etc/fstab とマウント状態の更新を行います。

In [30]:
!ansible -b -m mount -a 'name="{{{{volume_database_mount}}}}" \
    src="LABEL={{{{volume_database_fs_label}}}}" \
    opts={{{{volume_database_mount_opt}}}} \
    fstype={{{{volume_database_fs}}}} state=mounted' {target_group}

172.30.2.100 | SUCCESS => {
    "changed": true, 
    "dump": "0", 
    "failed": false, 
    "fstab": "/etc/fstab", 
    "fstype": "btrfs", 
    "name": "/opt/mysql", 
    "opts": "defaults", 
    "passno": "0", 
    "src": "LABEL=mysql"
}


/etc/fstabが更新されたことを確認します。

In [31]:
!ansible -a 'grep LABEL={{{{volume_database_fs_label}}}} /etc/fstab' {target_group}

172.30.2.100 | SUCCESS | rc=0 >>
LABEL=mysql /opt/mysql btrfs defaults 0 0



実際にマウントされていることを確認します。

In [32]:
!ansible -m shell -a 'mount | grep -e {{{{volume_database_mount}}}}' {target_group}

 [WARNING]: Consider using mount module rather than running mount

172.30.2.100 | SUCCESS | rc=0 >>
/dev/mapper/mysql on /opt/mysql type btrfs (rw,relatime,ssd,space_cache,subvolid=5,subvol=/)



## サブボリュームの作成

Moodle用ボリュームはコンテンツを格納するための領域とMoodleのPHPファイルを格納するための領域の両方の用途で利用します。それぞれの用途に応じたサブボリュームを作成します。

In [33]:
if 'btrfs' == load_group_var(target_group, 'volume_moodle_fs'):
    # コンテンツを格納する領域のサブボリュームを作成する
    !ansible -b -a 'btrfs subvolume create {{{{moodle_dir_contents}}}}' {target_group}
    # PHPファイルを格納する領域のサブボリュームを作成する
    !ansible -b -a 'btrfs subvolume create {{{{moodle_dir_scripts}}}}' {target_group}

172.30.2.100 | SUCCESS | rc=0 >>
Create subvolume '/opt/moodle/data'

172.30.2.100 | SUCCESS | rc=0 >>
Create subvolume '/opt/moodle/php'



作成されたことを確認します。

In [34]:
if 'btrfs' == load_group_var(target_group, 'volume_moodle_fs'):
    !ansible -b -a 'btrfs subvolume list -ta {{{{volume_moodle_mount}}}}' {target_group}

172.30.2.100 | SUCCESS | rc=0 >>
ID	gen	top level	path	
--	---	---------	----	
257	8	5		data
258	9	5		php

